In [2]:
import tensorflow as tf
import tensorflow.python.keras
#import keras
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import scipy
#from scipy.spatial.distance import directed_hausdorff, cosine
from IPython import display

2.0.0-beta1


## Utils function

**Create dataframe function**

In [3]:
def create_dataframe(path):
    abs_path = os.path.join(path, 'items.json' )
    print(abs_path)
    df_artworks = pd.read_json(abs_path)
    df_artworks = df_artworks.set_index('id')
    df_artworks = df_artworks.sort_index()
    df_artworks = df_artworks.reset_index()
    #clean data
    df_artworks['data'] = df_artworks['data'].str.replace('\r','')
    return df_artworks

**Encode artwork function**

In [4]:
from arts_preprocess_utils import get_image

def encode_artworks(x, encoder, base_img_path):

    try:
        img_path = os.path.join(base_img_path, str(x)+'.jpg')
        # Images
        image = get_image(img_path, img_Height=img_Height, img_Width=img_Width)

        # Encode
        code = encoder.predict(image)
        
        return code.reshape((-1,))

    except:
        print("Problem with file " + x)
        return None


## Load encoder

In [5]:
from tensorflow.python.keras.models import load_model

autoencoder_model = load_model('/root/work/trained_models/denoisy_encoder.h5')

## Load artwork's metadata

In [6]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'

#According to the dataset input used during autoencoder training
img_Height = 128
img_Width = 128
chanels= 3
N_CLASSES = 1

In [8]:
#List museums
for folder in os.listdir(BASE_PATH):
    museum_path = os.path.join(BASE_PATH, folder)
    #List tours inside the museum
    for tour_folder in os.listdir(museum_path):
        try:
            #Load tour data
            tour_path = os.path.join(museum_path, tour_folder)
            df = create_dataframe(tour_path)
            #Save data in csv format
            df.to_csv(os.path.join(tour_path, 'metadata.csv' ), index=False)

            #Encode artwork's tour
            base_img_path = os.path.join(tour_path, 'images' )
            df['code'] = df['id'].apply(encode_artworks, encoder=autoencoder_model, base_img_path=base_img_path)

            #Save code matrix
            encodes_list = list(df['code'])
            encode_matrix = np.stack(encodes_list)
            np.save(os.path.join(tour_path, 'code_matrix' ), encode_matrix)
        except:
            print("Problem with folder " + tour_folder)

/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_21/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_23/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_15/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_3/items.json
Problem with folder tour_3
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_2/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_0/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_20/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_22/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_5/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_6/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_11/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_14/items.json
/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_13/i

In [16]:
df = pd.read_csv('/root/work/datasets/artwork_sequence/rijksmuseum_crawler/tour_14/metadata.csv')
df.head()

,id,author,data,image_url,title
0,1,pieter claesz.,1627,https://lh3.googleusercontent.com/uqHsSOyPEFvu...,still life with a turkey pie
1,2,jan havicksz. steen,1655,https://lh3.googleusercontent.com/fLsffYqD3Ex7...,adolf and catharina croeser known as the burgo...
2,3,jan brueghel (ii),c. 1625 - c. 1630,https://lh3.googleusercontent.com/Z3PTxWguIfYH...,still life with flowers in a glass
3,4,jacob jordaens (i),c. 1635 - c. 1636,https://lh3.googleusercontent.com/iogmFpVbkXpz...,portrait of magdalena de cuyper
4,5,jacob coeman,1665,https://lh3.googleusercontent.com/-iNszdtfmRNt...,pieter cnoll cornelia van nijenrode and their ...
